In [1]:
!pip install transformers -q
!pip install torch -q

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import transformers
import numpy as np
import torch
from tqdm import notebook

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')
import re
from tqdm import notebook
import tqdm

In [13]:
data = pd.read_csv(r"C:\Users\nssavinov\Desktop\Пректы практикум\12. Машинное обучение для текстов\toxic_comments.csv")

In [15]:
batch_1 = data.sample(100)

In [17]:
X = batch_1['text']
y = batch_1['toxic']

In [19]:
X_clear = []

for i in X:
    lower_one = i.lower()
    clear_one = re.sub(r'^a-z', ' ', lower_one)
    clear_one = ' '.join(clear_one.split())

    X_clear.append(clear_one)

In [21]:
X = pd.Series(X_clear)

In [23]:
from transformers import BertTokenizer, BertForSequenceClassification

model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [24]:
tokenized = X.apply((lambda x: tokenizer.encode(x, add_special_tokens=True, max_length = 512, truncation=True)))

In [25]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

In [26]:
np.array(padded).shape

(100, 512)

In [27]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(100, 512)

In [28]:
#input_ids = torch.tensor(padded)
#attention_mask = torch.tensor(attention_mask)

batch_size = 100
embendings = []

for i in notebook.tqdm(range(padded.shape[0] // batch_size)):

    batch = torch.tensor(padded[batch_size*i:batch_size*(i+1)])

    attention_mask_batch = torch.tensor(attention_mask[batch_size*i:batch_size*(i+1)])

    with torch.no_grad():
        last_hidden_states = model(batch, attention_mask=attention_mask_batch)

    embendings.append(last_hidden_states[0][:,0,:].numpy())

  0%|          | 0/1 [00:00<?, ?it/s]

In [29]:
features = np.concatenate(embendings)

In [30]:
labels = batch_1['toxic']

In [31]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [32]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

LogisticRegression()

In [33]:
lr_clf.score(test_features, test_labels)

0.92